In [7]:
# importing the required libraries
import requests
import pandas as pd
import itertools
import os
import nltk
import aiohttp
import asyncio
import time
from collections import Counter
from openpyxl import load_workbook
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alokr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alokr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\alokr\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [9]:
# defining the urls for the web scraping
categories_url_list={'Electronics':"https://www.amazon.co.uk/s?k=electronics&crid=115Z61CWC2385&sprefix=electronics%2Caps%2C428&ref=nb_sb_ss_pltr-sample-20_1_11",
          'Fashion':"https://www.amazon.co.uk/s?k=fashion&i=fashion&crid=2CDEEX7MXZFKX&sprefix=fashion%2Cfashion%2C257&ref=nb_sb_noss_1",
          'Health & Personal Care':"https://www.amazon.co.uk/s?k=health&i=drugstore&crid=30X74E6JSK6WR&sprefix=heal%2Cdrugstore%2C895&ref=nb_sb_noss_2",
          'Home Acessories':"https://www.amazon.co.uk/s?k=home+accessories&crid=2N6847S0LOYY4&sprefix=home%2Caps%2C350&ref=nb_sb_ss_pltr-sample-20_4_4",
          'Books':"https://www.amazon.co.uk/s?k=best+books&i=stripbooks&crid=3PSB7LB026CA6&sprefix=best+books+%2Cstripbooks%2C234&ref=nb_sb_noss_2",
          'Toys and Games':"https://www.amazon.co.uk/s?k=toys&crid=9HN1VHEQQXCJ&sprefix=toys%2Caps%2C486&ref=nb_sb_noss_1"}

In [11]:
# displaying the urls
for i in categories_url_list.values():
    print(i)

https://www.amazon.co.uk/s?k=electronics&crid=115Z61CWC2385&sprefix=electronics%2Caps%2C428&ref=nb_sb_ss_pltr-sample-20_1_11
https://www.amazon.co.uk/s?k=fashion&i=fashion&crid=2CDEEX7MXZFKX&sprefix=fashion%2Cfashion%2C257&ref=nb_sb_noss_1
https://www.amazon.co.uk/s?k=health&i=drugstore&crid=30X74E6JSK6WR&sprefix=heal%2Cdrugstore%2C895&ref=nb_sb_noss_2
https://www.amazon.co.uk/s?k=home+accessories&crid=2N6847S0LOYY4&sprefix=home%2Caps%2C350&ref=nb_sb_ss_pltr-sample-20_4_4
https://www.amazon.co.uk/s?k=best+books&i=stripbooks&crid=3PSB7LB026CA6&sprefix=best+books+%2Cstripbooks%2C234&ref=nb_sb_noss_2
https://www.amazon.co.uk/s?k=toys&crid=9HN1VHEQQXCJ&sprefix=toys%2Caps%2C486&ref=nb_sb_noss_1


In [29]:
products_reviews_urls['Electronics']
products_reviews_urls['Fashion']
products_reviews_urls['Health & Personal Care']
products_reviews_urls['Home Acessories']
products_reviews_urls['Books']
products_reviews_urls['Toys and Games']

['https://www.amazon.co.uk/Misfittens-03936-Surprise-Stuffed-Collectable/dp/B09XXZHX52/ref=ice_ac_b_dpb',
 'https://www.amazon.co.uk/Misfittens-03936-Surprise-Stuffed-Collectable/dp/B09XXZHX52#customerReviews',
 'https://www.amazon.co.uk/Mattel-Extra-Terrestrial-Anniversary-Collectors-HHX97/dp/B09NZJK9T5#customerReviews',
 'https://www.amazon.co.uk/Airplane-Launcher-Glider-Outdoor-Aeorplane/dp/B0BXX19ZLT#customerReviews',
 'https://www.amazon.co.uk/gp/goldbox/',
 'https://www.amazon.co.uk/Tipao-Microphones-Portable-Machines-Birthday/dp/B0CM3BSDCX#customerReviews',
 'https://www.amazon.co.uk/Desire-Deluxe-Construction-Creativity-Educational/dp/B07B4GG659#customerReviews',
 'https://www.amazon.co.uk/RLINGX-3D-Pin-Art-Toy/dp/B09ZXZLBQM#customerReviews',
 'https://www.amazon.co.uk/Sago-Brothers-Modelling-Accessories-Hardening/dp/B07ZR4KX6Q#customerReviews',
 'https://www.amazon.co.uk/DEVRNEZ-Bluetooth-Microphone-Halloween-Christmas/dp/B09CCVM7SX#customerReviews',
 'https://www.amazon.co.uk

In [15]:
# function to get the product reviews
def get_product_reviews(url):
     res=requests.get(url)
     soup=BeautifulSoup(res.content,'html.parser')
     content=soup.find_all(class_="a-expander-content reviewText review-text-content a-expander-partial-collapse-content")
     content=[word_tokenize(i.span.text) for i in content if i and i.span]
     text=list(itertools.chain(*content))
     return text

In [17]:
# function to clean the reviews
def clean_products_reviews(reviews):
    tokens=word_tokenize(reviews)
    stop_words=set(stopwords.words('english'))
    filtered_tokens=[token for token in tokens if token.lower() not in stop_words]
    return filtered_tokens  

In [52]:
# text pre-processsing pipeline
vectorizer=CountVectorizer()
sid=SentimentIntensityAnalyzer()
def clean_and_convert_to_number(value):
    try:
        cleaned_value = value.split("+")[0].replace('K', '000').replace('M', '000000').replace('B', '000000000')
        return int(cleaned_value)
    except ValueError:
        return 0
    
def text_processing_pipeline(words):
#     vector=vectorizer.fit_transform(words)
    text=" ".join(words)
    polarity_score=sid.polarity_scores(text)
    if polarity_score['pos']>polarity_score['neg']:
        sentiment='positve'
    else:
        sentiment='negative'
    return sentiment  

def get_product_sentiment(products_reviews_url):
    words = get_product_reviews(url)
    words=" ".join(words)
    cleaned_reviews=clean_products_reviews(words)
    sentiment=text_processing_pipeline(cleaned_reviews)
    product_sentiment.append(sentiment)
    return product_sentiment    


In [54]:
for category,url_list in products_reviews_urls.items():
    product_sentiment=[]
    print(category)
    for index,url in enumerate(url_list):
        print(f"Fetching product {index+1} reviews")
        sentiment=get_product_sentiment(url)
        product_sentiment.append(sentiment)
        product_sentiments[category]=product_sentiment

Electronics
Fetching product 1 reviews
Fetching product 2 reviews
Fetching product 3 reviews
Fetching product 4 reviews
Fetching product 5 reviews
Fetching product 6 reviews
Fetching product 7 reviews
Fetching product 8 reviews
Fetching product 9 reviews
Fetching product 10 reviews
Fetching product 11 reviews
Fetching product 12 reviews
Fetching product 13 reviews
Fetching product 14 reviews
Fetching product 15 reviews
Fetching product 16 reviews
Fetching product 17 reviews
Fetching product 18 reviews
Fetching product 19 reviews
Fetching product 20 reviews
Fetching product 21 reviews
Fetching product 22 reviews
Fetching product 23 reviews
Fetching product 24 reviews
Fetching product 25 reviews
Fetching product 26 reviews
Fetching product 27 reviews
Fetching product 28 reviews
Fetching product 29 reviews
Fetching product 30 reviews
Fetching product 31 reviews
Fetching product 32 reviews
Fetching product 33 reviews
Fetching product 34 reviews
Fetching product 35 reviews
Fetching product 

In [21]:
# extracting the titles, prices, ratings, reviews, most recent orders and product sentiment
titles={}
prices={}
ratings={}
order_reviews={}
recent_orders={}
products_reviews_urls={}
products_reviews_texts={}
product_sentiments={}
for category,url in categories_url_list.items():
    title=[]
    price=[]
    rating=[]
    order_review=[]
    recent_order=[]
    res=requests.get(url)
    print(res)
    page_content=res.content
    soup=BeautifulSoup(page_content,'html.parser')
    if category=='Books':
        title=soup.find_all(class_='a-size-medium a-color-base a-text-normal')
    else:
        title=soup.find_all(class_='a-size-base-plus a-color-base a-text-normal')
    title=[i.text.split(",")[0] for i in title]
    price=soup.find_all(class_='a-price-whole')
    price=[int(i.text.replace(",","").split(".")[0])*91 for i in price]
    rating=soup.find_all(class_='a-icon-alt')
    rating=[float(i.text.split(" ")[0]) for i in rating]
    order_review=soup.find_all(class_='a-size-base s-underline-text')
    order_review=[int(i.text.replace(" ","").replace(",","")) for i in order_review]
    recent_order=soup.find_all(class_="a-size-base a-color-secondary")
    recent_order=[clean_and_convert_to_number(i.text) for i in recent_order if clean_and_convert_to_number(i.text)!=0]
    titles[category]=title
    prices[category]=price
    ratings[category]=rating
    order_reviews[category]=order_review
    recent_orders[category]=recent_order
    products_reviews_url=soup.find_all(class_='a-link-normal s-underline-text s-underline-link-text s-link-style')
    products_reviews_url=['https://www.amazon.co.uk'+i['href'] for i in products_reviews_url]
    products_reviews_urls[category]=products_reviews_url
    # product_sentiment=get_product_sentiment(products_reviews_url)
    # save_in_excel(category,title,price,rating,order_review,recent_order)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [18]:
# save the generated output in an excel sheet
directory="Products"
os.makedirs("Products",exist_ok=True)
def save_in_excel(category,title,price,ratings,order_reviews,recent_orders):
    file_name=category+".xlsx"
    path=os.path.join(directory,file_name)
    series1 = pd.Series(title, name="Title")
    series2 = pd.Series(price, name="Price")
    series3 = pd.Series(ratings, name="Ratings")
    series4 = pd.Series(order_reviews, name="Reviews")
    series5 = pd.Series(recent_orders, name="Recent Orders")
    df = pd.concat([series1, series2, series3, series4, series5], axis=1)
    with pd.ExcelWriter(path, engine='openpyxl') as writer:
        df.to_excel(writer,sheet_name=category,index=False)

In [31]:
print("Success")

Success
